In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from cleansummary import CleanSummary
plt.style.use('ggplot')
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_columns=None
pd.options.display.max_rows=None
pd.set_option('display.max_colwidth', None)

# warnings
import warnings
warnings.filterwarnings('ignore')

In [8]:
train = pd.read_csv('train.csv', low_memory=False)
train.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.84,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3,7,11.27,4.00,_,809.98,26.82,22 Years and 1 Months,No,49.57,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",-1,NaN,11.27,4.00,Good,809.98,31.94,NaN,No,49.57,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3,7,_,4.00,Good,809.98,28.61,22 Years and 3 Months,No,49.57,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",5,4,6.27,4.00,Good,809.98,31.38,22 Years and 4 Months,No,49.57,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.84,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",6,NaN,11.27,4.00,Good,809.98,24.80,22 Years and 5 Months,No,49.57,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [9]:
train.shape

(100000, 28)

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [5]:
columns_to_convert = ['Age','Num_of_Loan','Num_of_Delayed_Payment','Changed_Credit_Limit','Outstanding_Debt',
     'Amount_invested_monthly','Monthly_Balance','Annual_Income']

train[columns_to_convert] = train[columns_to_convert].astype(str)

for column in columns_to_convert:
    # Remove "_" from values and convert to floating-point numbers
    train[column] = train[column].str.extract(r'(-?\d+(\.\d+)?)')[0].astype(float)

# Convert the columns to integers
train[columns_to_convert] = train[columns_to_convert].astype(float)
train.head(3)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23.00,821-00-0265,Scientist,19114.12,1824.84,3,4,3,4.00,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3,7.00,11.27,4.00,_,809.98,26.82,22 Years and 1 Months,No,49.57,80.42,High_spent_Small_value_payments,312.49,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23.00,821-00-0265,Scientist,19114.12,NaN,3,4,3,4.00,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",-1,NaN,11.27,4.00,Good,809.98,31.94,NaN,No,49.57,118.28,Low_spent_Large_value_payments,284.63,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500.00,821-00-0265,Scientist,19114.12,NaN,3,4,3,4.00,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3,7.00,NaN,4.00,Good,809.98,28.61,22 Years and 3 Months,No,49.57,81.70,Low_spent_Medium_value_payments,331.21,Good


In [6]:
# Extract the number of years and months from the 'Credit_History_Age' column
years = train['Credit_History_Age'].str.extract('(\d+) Years?')
months = train['Credit_History_Age'].str.extract('(\d+) Months?')

# Convert the extracted years and months to numeric values
train['Years'] = pd.to_numeric(years[0], errors='coerce')
train['Months'] = pd.to_numeric(months[0], errors='coerce')
# Calculate the total months
train['Credit_History_Months'] = train['Years'] * 12 + train['Months']
train.drop(columns=['Credit_History_Age','Years','Months'], inplace=True)
train.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Credit_History_Months
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23.00,821-00-0265,Scientist,19114.12,1824.84,3,4,3,4.00,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3,7.00,11.27,4.00,_,809.98,26.82,No,49.57,80.42,High_spent_Small_value_payments,312.49,Good,265.00
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23.00,821-00-0265,Scientist,19114.12,NaN,3,4,3,4.00,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",-1,NaN,11.27,4.00,Good,809.98,31.94,No,49.57,118.28,Low_spent_Large_value_payments,284.63,Good,NaN
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500.00,821-00-0265,Scientist,19114.12,NaN,3,4,3,4.00,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3,7.00,NaN,4.00,Good,809.98,28.61,No,49.57,81.70,Low_spent_Medium_value_payments,331.21,Good,267.00
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23.00,821-00-0265,Scientist,19114.12,NaN,3,4,3,4.00,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",5,4.00,6.27,4.00,Good,809.98,31.38,No,49.57,199.46,Low_spent_Small_value_payments,223.45,Good,268.00
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23.00,821-00-0265,Scientist,19114.12,1824.84,3,4,3,4.00,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",6,NaN,11.27,4.00,Good,809.98,24.80,No,49.57,41.42,High_spent_Medium_value_payments,341.49,Good,269.00


In [ ]:
cs = CleanSummary(train)